## <span style="color:green"><h1><center>HPC TOOL: DEM Accessing & Processing (Single Job)</center></h1></span>
<center>Prepared by <br>
    <b>Rajesh Kalyanam*, Jibin Joseph^, and Venkatesh Merwade^</b><br> 
* Rosen Center for Advanced Computing, Purdue University<br> 
^ Lyles School of Civil Engineering, Purdue University<br> 
vmerwade@purdue.edu<br>
<b><br>
    FAIR Science in Water Resources</b><br></center>


## <span style="color:green">Objective</span>
<p style='text-align: justify;'> The objective of this tutorial is to perform the accessing and processing of DEM data for larger watersheds. Once you input the site ID and resolution, the tutorial will output clipped DEM raster for any regions across CONUS.</p> 

In [ ]:
import cybergis_compute_client
from cybergis_compute_client import CyberGISCompute
from IPython.display import Image, display, HTML
import os
from pynhd import NLDI
import matplotlib.pyplot as plt

## <span style="color:green">Step 1: Define the variables</span> i>

In [ ]:
## WRITE YOUR CODE
site_id = '' # Mississippi River at St. Paul, MN
resolution = '' # "1 for 1 arc-second, 13 for 1/3 arc-second"

In [ ]:
params_dem_process = {"site_id": site_id,
                    "resolution": resolution}
params_dem_process

In [ ]:
watershed=NLDI().get_basins(site_id,fsource='nwissite')

## Transform to Albers Equal Area projection (EPSG:5070)
watershed_albers = watershed.to_crs(epsg=5070)
## Calculate the area in square miles
## 1 square meter = 0.386102 square miles
watershed_albers['area_sq_mi'] = watershed_albers.area / 1e6 * 0.386102  
#print(watershed_albers['area_sq_mi'][0])

## Plot the watershed
## DD indicates latitude/ longitude degrees is in decimal
ax = watershed.plot(facecolor="b", 
                    edgecolor="k", 
                    figsize=(8, 8))
plt.title(f"Watershed Shapefile in {watershed.crs} Projected CRS\n(USGS:{site_id}, "+
          f"Area = {round(watershed_albers['area_sq_mi'].iloc[0],2)} sq. mi.)")
plt.xlabel("Longitude (DD)")
plt.ylabel("Longitude (DD)")

## <span style="color:green">Step 2: HPC Job Submission</span>

<h4 style="color:red;"> User Interaction Required </h4>

- Run the cell below
- Set Time(Minutes) to 30 and cpus_per_task to 128
- Click on "Submit Job" on the "Job Configuration" tabpage 
- Wait until Job is finished (2-3 mins)
- Switch to "Download Job Result" tabpage
- Choose "/preview" and click on Download
- Wait until downloading is finished
- Proceed to the next cell

In [ ]:
cybergis = CyberGISCompute(url="cgjobsup.cigi.illinois.edu", isJupyter=True, protocol="HTTPS", port=443, suffix="v2")
cybergis.show_ui(defaultJob="Watershed_DEM_Processing", input_params=params_dem_process)

## <span style="color:green">Step 3: Plotting the clipped DEM (preview)</span>

In [ ]:
clipped_preview_output = cybergis.recentDownloadPath
clipped_preview_file = os.path.join(clipped_preview_output, f'clipped_raster_{site_id}.png')
print(clipped_preview_file)
if not os.path.isfile(clipped_preview_file):
    display(HTML('<h4 style="color:red;">It appears you did not download the job results per instruction above, please double check!</h4>'))
Image(filename=clipped_preview_file) 

## DONE